# Lampy White Paper

**Big Questions**

1. What to expect from Lampy? What not to expect (and probably use the concepts from Tensorflow / Numpy / Thrust / etc. )?
2. Final goal for the semester.


**What people already have?**

- [Tensorflow: Partitioning and Scheduling](https://www.tensorflow.org/guide/extend/architecture): provide the architecture of how tensorflow do massive computing on mobile and cloud devices. It _includes_ mobile devices, IoT network. 

**What Lampy should do now?**
1. **Better scheduling.**. More flexible to control Lambda resources. Lampy provide expected computation time / computation resource requirement to provision the size of computation. The infrastructure can utilize the information and potentially boost the information. Acquiring information from application and notify the insfrastructure helps us to potentially pre-heat the Lambdas so as to avoid cold start. 
2. **Tolerance of failure.** What should we do when one lambda failed? How should application communicate with OpenLambda and relaunch the job? 
99. (Not this semester) Optimization of computation on Edge Device.

**Final Goal**
1. Build speech diarazation algorithm (Fisher Linear Semi-Discriminant Analysis Algorithm) on Lampy.
2. Compare tensorflow and Lampy (and see how to win?)

# LamPy - Development Log

## Short-term Goal 

1. Wrapper: A wrapper of `numpy`
2. Scheduling: Take over the scheduling of calculation in the cloud

## Thinkings of Development

This little `Lampy` use `+` operator as an example and show how we want to develop the Lampy idea.

### Version 1: Support very simple calculation.

In [145]:
import numpy as np

In [207]:
class LamObject():
    def __init__(self, val=None):
        if isinstance(val, numpy.ndarray):
            self.val = val
        else:
            self.val = np.array(val)
        
    def __add__(self, obj):
        print(f"Doing {self.val} + {obj.val}")
        return LamObject(self.val + obj.val)
    
    def __str__(self):
        return self.val.__str__()

    def __repr__(self):
        return self.val.__str__()

In [208]:
a = LamObject([1,2,3])
b = LamObject([1,2,3])
c = a + b

Doing [1 2 3] + [1 2 3]


In [209]:
c

[2 4 6]

### Version 2: Create a graph diagraph of the Lampy object

In [385]:
class LamObject():
    def __init__(self, val=None, children=[]):
        self.children = children   # [LamObject]
        if isinstance(val, numpy.ndarray):
            self.val = val
        else:
            self.val = np.array(val)
        
    def run(self):
        if not len(self.children):
            return self.val
        for i in self.children:
            i.run()
        sum = self.children[0].val
        for i in self.children[1:]:
            sum += i.val
        self.val = sum
        return sum
        
        
    def __add__(self, obj):
        return LamObject(val=None, children=[self, obj])
    
    def __str__(self):
        return self.val.__str__()

#     def __repr__(self):
#         return self.val.__str__()

In [393]:
a = LamObject([1,2,3])
b = LamObject([1,2,3])
d = LamObject([4])

c = a + b + d

In [394]:
c.val, c.children

(array(None, dtype=object),
 [<__main__.LamObject at 0x10e776550>, <__main__.LamObject at 0x10e7764e0>])

In [395]:
c.run()

array([ 6,  8, 10])

In [396]:
c.val

array([ 6,  8, 10])

## Version 3: Scheduling

As a prototype, we manually configurate the array and make it split in to certain size.
For example, we have two matrix $a$, $b$ that goes:

$$
a = \begin{bmatrix} 
1 & 2 & 3 & 4\\ 
1 & 2 & 3 & 4\\ 
1 & 2 & 3 & 4\\
1 & 2 & 3 & 4
\end{bmatrix}
,
b = \begin{bmatrix} 
1 & 2 & 3 & 4\\ 
1 & 2 & 3 & 4\\ 
1 & 2 & 3 & 4\\
1 & 2 & 3 & 4
\end{bmatrix}
$$

Then we split them intpu four sub matrices, each 2-by-2, and then schedule the operation (in local environment, for now -- we will do the I/O and remote scheduling a bit later)

In [415]:
# ############
# Haven't done with the functionality
#############
class LamObject():
    def __init__(self, val=None, children=[], config=None):
        self.children = children   # [LamObject]
        self.config = config       # Configuration Inforamtion
        if isinstance(val, numpy.ndarray):
            self.val = val
        else:
            self.val = np.array(val)
        
    def size(self):
        return self.val.shape

    def run(self):
        if not len(self.children):
            return self.val
        for i in self.children:
            i.run()
        sum = self.children[0].val
        for i in self.children[1:]:
            sum += i.val
        self.val = sum
        return sum
    
    def config(submatrix=None):
        """
        # In this implementation we assume:
        #   1. We know what our children's size are prior computation by calling `val.shape`.
        #      (This point can be refined by specifying pre-defined input of the shape, or let's think of other methods to dynamically do this )
        #   2. We don't care about the fact that the splitting of matrix in memory will incur overhead. We will try to solve it in the next try
        """
        pass
#         if submatrix is None:
#             return
            

        
        
    def __add__(self, obj):
        return LamObject(val=None, children=[self, obj])
    
    def __str__(self):
        return self.val.__str__()

#     def __repr__(self):
#         return self.val.__str__()

In [414]:
a = LamObject([
[1,2,3,4],
[1,2,3,4],
[1,2,3,4],
[1,2,3,4]
])
b = LamObject([
[1,2,3,4],
[1,2,3,4],
[1,2,3,4],
[1,2,3,4]
])

c = a + b

In [401]:
c.config(submatrix=[2,2]) # Run with 2x2 submatrix
c.run()

array([[2, 4, 6, 8],
       [2, 4, 6, 8],
       [2, 4, 6, 8],
       [2, 4, 6, 8]])

In [406]:
a = np.array([
[1,2,3,4],
[1,2,3,4],
[1,2,3,4],
[1,2,3,4]
])

In [412]:
a.shape

(4, 4)

## Version 4: Delay Input